In [6]:
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
import warnings
from keras.preprocessing import image
import cv2
import numpy as np
import glob
import re
from datetime import datetime as dt
from scipy.spatial import distance
from imutils import face_utils
from keras.models import load_model
import tensorflow as tf
from collections import Counter
import pandas as pd
from fr_utils import *
from inception_blocks_v2 import *

#with CustomObjectScope({'tf': tf}):
FR_model = load_model('nn4.small2.v1.h5')
print("Total Params:", FR_model.count_params())
print("Summary:", FR_model.summary())

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

threshold = 0.25

face_database = {}

for roll in os.listdir('images'):
    for image in os.listdir(os.path.join('images',roll)):
        identity = roll + '--' + os.path.splitext(os.path.basename(image))[0]
        face_database[identity] = fr_utils.img_path_to_encoding(os.path.join('images',roll,image), FR_model)

print(face_database)

df = pd.DataFrame(columns=['Roll No', 'Name', 'Attendance', 'Out Time', 'In Time'])
l1 = []
l2 = []

video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
        roi = frame[y:y+h, x:x+w]
        encoding = img_to_encoding(roi, FR_model)
        min_dist = 100
        identity = None

        for(name, encoded_image_name) in face_database.items():
            dist = np.linalg.norm(encoding - encoded_image_name)
            if(dist < min_dist):
                min_dist = dist
                identity = name
            #print('Min dist: ',min_dist)

        if min_dist < 0.1:
            cv2.putText(frame, "Face : " + identity[:-1], (x, y - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.putText(frame, "Dist : " + str(min_dist), (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            l1.append(" ".join(re.findall("[a-zA-Z]+", identity)))
            l2.append(identity.split('--')[0])
        else:
            cv2.putText(frame, 'No matching faces', (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)

        if len(l2)==100:
            entry = Counter(l1).most_common(1)[0][0]
            roll_no = Counter(l2).most_common(1)[0][0]
            time=dt.now().strftime("%I:%M:%S %p")


            if roll_no not in df['Roll No'].values:
                df.loc[len(df)] = [roll_no, entry, 'OUT',time,"NOT IN HOSTEL"]
            else:
                df.loc[df['Roll No']==roll_no,['Attendance','In Time']] = ['IN',time]

            print("Entry added, you may leave now!")
            
            cv2.waitKey(5000)
            l1=[]
            l2=[]

    cv2.imshow('Face Recognition System', frame)
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break

df.to_csv('result.csv', index=False)

video_capture.release()
cv2.destroyAllWindows()


Total Params: 3743280
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
_______________________________________________________________________________________

{'aman--aman1': array([[ 9.38057005e-02, -5.39511722e-03, -3.37287374e-02,
        -4.14344966e-02,  6.77882554e-03,  2.03173801e-01,
        -1.46483257e-01,  4.19050269e-03, -1.96486749e-02,
        -6.85242563e-02,  9.27217975e-02,  9.91271343e-03,
         3.63313081e-03,  6.08656630e-02,  1.11388110e-01,
         1.50991827e-01, -1.90789327e-02,  2.19492931e-02,
        -8.42026994e-03, -1.19661927e-01, -4.23372462e-02,
         3.58180963e-02, -6.51732534e-02, -1.36396334e-01,
         8.26627668e-03, -8.11794586e-03,  1.10797368e-01,
        -9.63408723e-02,  1.54765761e-02,  4.47824262e-02,
         1.49178565e-01,  3.67943905e-02,  1.26577944e-01,
         5.24431281e-02,  5.84069304e-02, -1.27283007e-01,
         2.76963226e-02,  1.20391436e-01,  4.56606336e-02,
        -1.68795642e-02,  8.99098348e-03,  7.49406144e-02,
         2.66109016e-02, -1.09652849e-02,  1.30710369e-02,
         1.48936510e-01,  2.34387293e-02, -1.22635961e-01,
        -5.62533066e-02,  5.37943430e-02